## Bayesian Sets Recommender System - NPM

In [ ]:
# Algorithm Implementation: https://github.com/MaLL-UFSCar/bayessets
# Research Paper: 

In [57]:
!pip install git+https://github.com/MaLL-UFSCar/bayessets.git

  Cloning https://github.com/MaLL-UFSCar/bayessets.git to /tmp/pip-req-build-kdfeqxr0
  Running command git clone -q https://github.com/MaLL-UFSCar/bayessets.git /tmp/pip-req-build-kdfeqxr0
  Stored in directory: /tmp/pip-ephem-wheel-cache-liib3hja/wheels/17/99/fe/bc3c266fdef546aaf825421da93f0826c5eb36802c60c7247e
Successfully built BayesSets
You should consider upgrading via the 'pip install --upgrade pip' command.


In [58]:
!pip show -f BayesSets

Name: BayesSets
Version: 0.2
Summary: Implementation of the Bayesian Sets algorithm and variants
Home-page: https://github.com/MaLL-UFSCar/bayessets
Author: Diorge Brognara
Author-email: diorge.bs@gmail.com
License: MIT
Location: /home/antrived/anaconda3/lib/python3.6/site-packages
Requires: numpy, scipy
Required-by: 
Files:
  BayesSets-0.2.dist-info/INSTALLER
  BayesSets-0.2.dist-info/METADATA
  BayesSets-0.2.dist-info/RECORD
  BayesSets-0.2.dist-info/WHEEL
  BayesSets-0.2.dist-info/top_level.txt
  bayessets/__init__.py
  bayessets/__pycache__/__init__.cpython-36.pyc
  bayessets/__pycache__/bayessets.cpython-36.pyc
  bayessets/bayessets.py


In [141]:
import numpy
import numpy as np
import pandas as pd
from scipy import sparse
import itertools

In [60]:
# check working directory
import os
os.getcwd()

'/home/antrived/Dropbox/Redhat_Work/PycharmProjects/10-Bayesian-Sets-Recom-NPM'

### Data Loading & Pre-processing

In [61]:
# place 'package-to-id-dict-without-trans.json' and 'manifest-to-id-without-trans.pickle'
    # in wd from 'aagshah-pypi-insights' bucket

In [62]:
def cal_sparsity(array):
    num_total = total_elems(array)
    num_non_zero = zero_elems(array, num_total)
    sparsity = num_non_zero/num_total
    print("Sparsity of matrix is = {}".format(sparsity))
    return sparsity

def zero_elems(array, num_total):
    non_zero = numpy.count_nonzero(array)
    return num_total-non_zero

def total_elems(array):
    shape = array.shape
    return shape[0]*shape[1]

In [63]:
import json
import pickle

In [29]:
with open('package_to_index_map.json', 'r') as f:
    pack_to_id = json.load(f)
print(len(pack_to_id)) # 29704
pack_to_id

29704


{'pon-logger': 1,
 '@ilg/cli-start-options': 11,
 'node-red-contrib-jsonfilter': 8615,
 'phantom-pool': 6,
 'normalize-css-color': 9122,
 'has-own-prop': 8,
 'blgr': 10,
 'iterable-findindex': 12,
 'oss-client': 13,
 'tiny-uuid': 14,
 'babel-plugin-react-transform': 16,
 'run-callback': 22306,
 'moment-es6': 22,
 'prfnw': 26,
 'apeman-proto-infr': 27,
 'pym.js': 28,
 'rst-selector-parser': 29,
 'desandro-matches-selector': 212,
 'octobus-crud': 35,
 'topcoat-notification': 38,
 'co-express': 39,
 'postcss-css-variables': 40,
 'falcor-http-datasource': 42,
 'wednesday': 43,
 'blockcypher-unofficial': 45,
 'xutil': 47,
 'asyncer.js': 305,
 'grapesjs-navbar': 50,
 'is-array-like-x': 51,
 'ansi-yellow': 52,
 'xmlrpc-lite': 56,
 'livedoc': 59,
 'xml-stream': 60,
 'ticketfly-css-text-utilities': 19560,
 'errors': 65,
 'ls-r': 68,
 'node-red-node-notify': 69,
 'ip-to-int': 71,
 'spirit-middleware': 72,
 'draft-js-hashtag-plugin': 73,
 'numeral': 74,
 'isbin': 76,
 '@unction/istype': 78,
 '@pr

In [51]:
temp=pd.read_csv("manifest_user_data.dat",sep=" ::",header=None)
# checked separately, 29704 unique IDs in the 0 column of temp
man_to_id = {}
for i in range(len(temp)):
    list1 = []
    for j in temp[0][i].split():
        list1.append(int(j))
    man_to_id[str(i)] = list1
man_to_id # 28835

/home/antrived/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


{'0': [7, 22986, 23137, 9848, 20807, 14540, 6885, 7254],
 '1': [6, 13342, 10700, 22836, 9543, 4362, 9994],
 '2': [5, 6885, 11106, 10215, 20807, 17452],
 '3': [5, 24566, 6885, 11106, 10215, 18656],
 '4': [13,
  26809,
  18159,
  22986,
  8074,
  9371,
  1871,
  1362,
  1166,
  23906,
  20807,
  1265,
  6885,
  9512],
 '5': [5, 13378, 1362, 13342, 27376, 20807],
 '6': [6, 15077, 14404, 9371, 27583, 8624, 10579],
 '7': [5, 17190, 14774, 15406, 16594, 29063],
 '8': [8, 24619, 28619, 18105, 10215, 11106, 19288, 6885, 2907],
 '9': [5, 22241, 28239, 23906, 14200, 19288],
 '10': [5, 1362, 15547, 23906, 27427, 1871],
 '11': [5, 23285, 9371, 2268, 20807, 26870],
 '12': [5, 10795, 5085, 19288, 9934, 1871],
 '13': [5, 1362, 16659, 11132, 23906, 3289],
 '14': [5, 5646, 19288, 3143, 6102, 26870],
 '15': [5, 6885, 11106, 1166, 23906, 1871],
 '16': [5, 22512, 6912, 8156, 7957, 19288],
 '17': [5, 5999, 25106, 18185, 11762, 2920],
 '18': [9, 27033, 2380, 28805, 28620, 22186, 19288, 8063, 13845, 26107],


In [52]:
users = len(man_to_id)
print(users) # 28835
items = len(pack_to_id)
print(items) # 29704

28835
29704


In [54]:
# initialize user-item matrix
rating_matrix = numpy.zeros((users, items))

In [55]:
# populate user-item matrix
for user, item_list in man_to_id.items():
    for item in item_list:
        rating_matrix[int(user)][item] = 1
# view matrix
print(rating_matrix.shape[0])
print(rating_matrix.shape[1])
rating_matrix[:3] # gets first 3 rows

28835
29704


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [65]:
cal_sparsity(rating_matrix)
print(sum(sum(rating_matrix))) # no. of ratings = 231704.0

Sparsity of matrix is = 0.9997294804606071
231704.0


In [66]:
sparse_rating_matrix = sparse.csr_matrix(rating_matrix)
print(sparse_rating_matrix.shape[0]) # 
print(sparse_rating_matrix.shape[1]) # 

28835
29704


In [67]:
sparse_rating_matrix = sparse_rating_matrix.transpose()
print(sparse_rating_matrix.shape[0]) 
print(sparse_rating_matrix.shape[1])

29704
28835


In [68]:
sparse_rating_matrix[:10]

<10x28835 sparse matrix of type '<class 'numpy.float64'>'
	with 25170 stored elements in Compressed Sparse Column format>

### Training

In [69]:
import bayessets
model = bayessets.BernoulliBayesianSet(sparse_rating_matrix)

In [70]:
with open('Bayesian_Sets.pkl', 'wb') as f:
    pickle.dump(model, f)

In [71]:
with open('Bayesian_Sets.pkl', 'rb') as f:
    model = pickle.load(f)

### Scoring

In [808]:
import random
from math import ceil

def get_packages_from_id(package_ids):
    package_list = list()
    for i in package_ids:
        package = id_to_pack.get(str(i))
        package_list.append(package)
    return package_list

def map_input_to_package_ids(input_stack):
    package_id_list = list()
    for package in input_stack:
        package_id = pack_to_id.get(package)
        if package_id is not None:
            package_id_list.append(package_id)
    return package_id_list

# calculate probability of recommendatios
def calculate_recomm_prob(scores):
    
#     # use this to remove outliers, min/max values
#     scores_1 = np.setdiff1d(scores, min(scores))
#     scores_2 = np.setdiff1d(scores_1, max(scores_1))
#     normalized_scores=(scores-scores_2.min())/(scores.max()-scores_2.min())
    
    normalized_scores=(scores-scores.min())/(scores.max()-scores.min())
    Prob = []
    for index in recommendations:
        Prob.append(int(100*normalized_scores[index]))
    return Prob

In [73]:
# with open('id-to-package-dict-without-trans.json', 'r') as f:
#     id_to_pack = json.load(f)

In [74]:
# create 'id_to_pack' dict
id_to_pack = {}
for key, val in pack_to_id.items():
    id_to_pack[str(val)] = key

In [293]:
input_stack = ['abacus-demo-client','numeral','structured-cli','krl-parser' ]

In [884]:
str1 = str(
"""
"angular": "^1.4.9",
"angular-ui-router": "^0.2.17",
"body-parser": "^1.14.2",
"bootstrap-sass": "^3.3.6",
"express": "^4.13.4",
"jquery": "^2.2.0",
"kerberos": "0.0.18",
"lodash": "^4.1.0",
"mongoose": "^4.4.0"
"""
)

In [885]:
input_stack = []
for i in range( len(str1.split(":")) - 1 ):
    val = str1.split(":")[i].split("\n")[1]
    val = val.replace('"','')
    val = val.strip()
    input_stack.append(val)
input_stack

['angular',
 'angular-ui-router',
 'body-parser',
 'bootstrap-sass',
 'express',
 'jquery',
 'kerberos',
 'lodash',
 'mongoose']

In [886]:
input_stack = map_input_to_package_ids(input_stack)
print(input_stack)
# QC
print(get_packages_from_id(input_stack))

[19249, 15602, 9371, 5646, 2291, 19511, 16597]
['angular', 'bootstrap-sass', 'express', 'jquery', 'kerberos', 'lodash', 'mongoose']


In [887]:
# Scoring
scores = model.query(list(input_stack))
ranking = numpy.argsort(scores)[::-1]
top30 = ranking[:30]
recommendations = numpy.array(list(itertools.compress(top30,
    [i not in input_stack for i in top30])))
print(recommendations)
print(get_packages_from_id(recommendations))
print(calculate_recomm_prob(scores))

[    5 27016  6885     6 24307 10215 14540     7 21388     8  3421 22836
  7796  2218  1490     9  1362  1871  8156 25764   221 27994    10 11106
  6139]
['colors-promise', 'bluebird', 'async', 'phantom-pool', 'moment', 'request', 'debug', 'spirit-site-data', 'chalk', 'has-own-prop', 'yeoman-generator', 'q', 'fs-extra', 'yosay', 'mkdirp', 'mocha-runner', 'socket.io', 'commander', 'winston', 'bootstrap', 'uuid', 'react', 'blgr', 'colors', 'mongodb']
[16, 15, 14, 14, 12, 10, 9, 8, 8, 7, 6, 5, 5, 5, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3]


In [888]:
# sorted(scores, reverse=True)

In [889]:
recom = get_packages_from_id(recommendations)
probs = calculate_recomm_prob(scores)
recom_prob = []
for i, prob in enumerate(probs):
    if prob >= 25:
        recom_prob.append(str("{a}({b}%)".format( a=recom[i],b=probs[i] )))

recom_prob

[]